In [32]:
import pyodbc
import pandas as pd
import os 
import sqlite3 
import shutil
from sqlalchemy import create_engine
import urllib


# SQLAlchemy
# Database connection details
server = 'PRD-APP-PBI-GW\\UCRHEALTH_PROD'  
database = 'ucr_health'  
username = 'ucr_svc_analytics'  
password = 'Ucr53Rv!c3'


# Construct the connection string
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
encoded_conn_str = urllib.parse.quote_plus(connection_string)

# Create the SQLAlchemy engine 
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={encoded_conn_str}')

# Read query results directly into a DataFrame
#df = pd.read_sql('SELECT * FROM [ucr_health].[rpt].[appointment]', engine)
query = '''
select distinct 
	[dept_specialty_name]
	,[center_name]
	,[visit_provider_name]
	,[visit_prov_id]
from [ucr_health].[rpt].[appointment]
where fiscal_year >= (select max(fiscal_year) -3 from [ucr_health].[rpt].[appointment])
	and appt_status = 'Completed'
	and subdivision_name is null
	and visit_provider_name not like '%UCR%'
order by dept_specialty_name, visit_provider_name
'''
df = pd.read_sql(query, engine)

# Close the connection (optional with SQLAlchemy, but good practice)
engine.dispose() 

In [8]:
# connect to sqlite
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


production = sqlite3.connect('C:\\python_development\\productivity.db', timeout=30) 
#pd.read_sql_query('SELECT * FROM sqlite_master where name like "%pro%";', production)
# src_provider_activity
# src_provider_specialty

In [36]:
df.head()

,dept_specialty_name,center_name,visit_provider_name,visit_prov_id
0,CARDIOLOGY,UCR Citrus Tower,"LIN, BILLY",133367
1,GENERAL PSYCHIATRY,UCR Citrus Tower,"DUARTE, EDDER R",306535
2,GENERAL PSYCHIATRY,UCR Citrus Tower,"ESPINOZA, ALFRED",315344
3,GENERAL PSYCHIATRY,UCR Citrus Tower,"FLETCHER, CHANTEL",315275
4,GENERAL PSYCHIATRY,UCR Citrus Tower,"HADDAD, LANA",289751


In [40]:
# write temp table to sqlite for provider insert
df.to_sql('temp_provider_insert', production, if_exists='replace', index=False)

22

In [52]:
# psych = 'A02200', 1112
# card = 'A02214', 1007
# IM = 'A02090', 1035 OP 1029 IP
# neur = 'A02203, 1038
pd.read_sql_query('select * from src_provider_specialty limit 5', production)

,provider_name,provider_id,specialty,specialty_id
0,"AL-KHOURY, LAMA MICHEL",50471,Neurology,1038
1,"LAKIN, LYNSEY",57005,NP: Neurology,1178
2,"GOWDA, NANDINI NETKAL",269915,Internal Medicine: Ambulatory Only (No Inpatie...,1035
3,"FOROUHAR, SARA",265879,Pediatrics: Hospitalist,1089
4,"MUFTI, HOVEDA",97356,Hospitalist: Internal Medicine,1029


In [57]:
# create insert statements

insert = '''
insert into src_provider_specialty (provider_name, provider_id, specialty, specialty_id)
select
visit_provider_name as provider_name
, visit_prov_id as provider_id
, case 
  when dept_specialty_name = 'CARDIOLOGY' then 'Cardiology: Noninvasive'
  when dept_specialty_name = 'GENERAL PSYCHIATRY' then 'Psychiatry: General'
  when dept_specialty_name = 'INTERNAL MEDICINE' then 'Internal Medicine: Ambulatory Only (No Inpatient Work)'
  when dept_specialty_name = 'NEUROLOGY' then 'Neurology'
  end as specialty
, case 
  when dept_specialty_name = 'CARDIOLOGY' then 1007
  when dept_specialty_name = 'GENERAL PSYCHIATRY' then 1112
  when dept_specialty_name = 'INTERNAL MEDICINE' then 1035
  when dept_specialty_name = 'NEUROLOGY' then 1038
  end as specialty_id
from temp_provider_insert
'''
#pd.read_sql_query(insert, production)
production.execute(insert)
production.commit()

In [59]:
insert = '''
insert into src_provider_activity (provider_name, provider_id, activity_code, employee_id)
select
visit_provider_name as provider_name
, visit_prov_id as provider_id
, case 
  when dept_specialty_name = 'CARDIOLOGY' then 'A02214'
  when dept_specialty_name = 'GENERAL PSYCHIATRY' then 'A02200'
  when dept_specialty_name = 'INTERNAL MEDICINE' then 'A02090'
  when dept_specialty_name = 'NEUROLOGY' then 'A02203'
  end as activity_code
, null as employee_id
from temp_provider_insert
'''
#pd.read_sql_query(insert, production)
production.execute(insert)
production.commit()

In [67]:
pd.read_sql_query('select * from src_provider_activity where provider_id = 133367', production)

,provider_name,provider_id,activity_code,employee_id
0,"LIN, BILLY",133367,A02214,None


In [69]:
pd.read_sql_query('select * from vw_provider_dimension where provider_id = 133367', production)

,provider_name,provider_id,primary_specialty,primary_specialty_id,inpatient_specialty,inpatient_specialty_id,activity_code,activity_name,subdivision_code,subdivision_name,epic_specialty
0,"LIN, BILLY",133367,Cardiology: Noninvasive,1007,None,None,A02214,Cardiology and Interventional Cardio Prgm,D02020,Internal Medicine GME Dept,CARDIOLOGY
